In [1]:
import numpy as np
import pandas as pd
import sys, os

In [61]:
f_euclid_dist = lambda a,b: np.linalg.norm(a-b)

def f_h_step(x, a):
    return 1 if (x<=a) else 0

f_y = lambda k : -np.log10(k)

In [92]:
'''
Get the Y data
'''
path = 'C:/Users/beryl/Documents/Computational Science/Kanazawa/Thesis/Dataset/PP/index/INDEX_general_PP.2018'
mol_units = {'uM':1.e-6, 'pM':1.e-12, 'fM':1.e-15, 'nM':1.e-9, 'mM':1.e-3}

#load the index file
l = []
with open(path, 'r') as f:
    for line in f:
        if not line.startswith('#'):
            clean_line = line.rstrip()
            l.append((line.rstrip()).split())
df_idx = (pd.DataFrame(l)).rename(columns={0:'id',3:'k'})
#print(df_idx)

#generate the -log_10k values
op_tokens = ['=','~','>','<']
logys = np.zeros(df_idx.shape[0])
for i in range(df_idx.shape[0]):
    string = df_idx.loc[i]['k']
    for s in string:
        if s in op_tokens:
            split_str = string.split(s)
            break
    logys[i] = f_y( float(split_str[-1][:-2]) * mol_units[split_str[-1][-2:]] )
df_idx["log_y"] = logys

print(df_idx.loc[9])
print(len(df_idx))

id              2pcb
1               2.80
2               1993
k            Kd=10uM
4                 //
5           2pcb.pdf
6          (104-mer)
7         cytochrome
8                 c,
9        Ka=10^5M-1,
10          Kd=10-5M
11              None
12              None
13              None
14              None
15              None
16              None
17              None
18              None
19              None
20              None
21              None
22              None
23              None
24              None
25              None
26              None
27              None
28              None
29              None
30              None
31              None
32              None
33              None
34              None
35              None
36              None
37              None
log_y              5
Name: 9, dtype: object
2416


In [104]:
'''
complex files loader
'''
path = 'C:/Users/beryl/Documents/Computational Science/Kanazawa/Thesis/Dataset/PP'


complex_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
print(prot_files)
print(len(complex_files))
print(path+'/'+complex_files[0])

l =[]
with open(path+'/'+complex_files[0], 'r') as f:
    chain = []
    for line in f:
        if line.startswith('ATOM') or line.startswith('TER'):
            clean_line = (line.rstrip()).split()
            l.append(clean_line)
df_atoms = (pd.DataFrame(l)).rename(columns={6:'x_coor', 7:'y_coor', 8:'z_coor', 11:'atom_type'})
df_atoms

['1a22.ent.pdb', '1a2k.ent.pdb', '1a3b.ent.pdb', '1a4y.ent.pdb', '1acb.ent.pdb', '1ahw.ent.pdb', '1ak4.ent.pdb', '1akj.ent.pdb', '1an1.ent.pdb', '1atn.ent.pdb', '1ava.ent.pdb', '1avg.ent.pdb', '1avx.ent.pdb', '1avz.ent.pdb', '1axi.ent.pdb', '1ay7.ent.pdb', '1azz.ent.pdb', '1b27.ent.pdb', '1b2s.ent.pdb', '1b2u.ent.pdb', '1b3s.ent.pdb', '1b6c.ent.pdb', '1bj1.ent.pdb', '1bql.ent.pdb', '1brs.ent.pdb', '1bth.ent.pdb', '1buh.ent.pdb', '1bvk.ent.pdb', '1bvn.ent.pdb', '1c9p.ent.pdb', '1c9t.ent.pdb', '1ca0.ent.pdb', '1cbw.ent.pdb', '1clv.ent.pdb', '1cmx.ent.pdb', '1cn4.ent.pdb', '1cz8.ent.pdb', '1d6r.ent.pdb', '1de4.ent.pdb', '1dee.ent.pdb', '1dfj.ent.pdb', '1dhk.ent.pdb', '1dit.ent.pdb', '1djs.ent.pdb', '1dp5.ent.pdb', '1dpj.ent.pdb', '1dqj.ent.pdb', '1du3.ent.pdb', '1dzb.ent.pdb', '1e3u.ent.pdb', '1e4k.ent.pdb', '1e6e.ent.pdb', '1e6j.ent.pdb', '1e96.ent.pdb', '1eaw.ent.pdb', '1eer.ent.pdb', '1ees.ent.pdb', '1efn.ent.pdb', '1efx.ent.pdb', '1eja.ent.pdb', '1emv.ent.pdb', '1eo8.ent.pdb', '1es0.e

,0,1,2,3,4,5,x_coor,y_coor,z_coor,9,10,atom_type
0,ATOM,1,N,PHE,A,1,73.227,31.442,101.410,1.00,44.85,N
1,ATOM,2,CA,PHE,A,1,72.120,30.453,101.239,1.00,44.10,C
2,ATOM,3,C,PHE,A,1,71.047,30.634,102.304,1.00,45.17,C
3,ATOM,4,O,PHE,A,1,71.319,30.520,103.512,1.00,46.16,O
4,ATOM,5,CB,PHE,A,1,72.638,29.012,101.282,1.00,41.75,C
5,ATOM,6,CG,PHE,A,1,71.672,28.022,100.712,1.00,41.16,C
6,ATOM,7,CD1,PHE,A,1,71.557,27.858,99.343,1.00,40.67,C
7,ATOM,8,CD2,PHE,A,1,70.844,27.291,101.531,1.00,41.70,C
8,ATOM,9,CE1,PHE,A,1,70.635,26.989,98.805,1.00,39.17,C
9,ATOM,10,CE2,PHE,A,1,69.915,26.416,100.989,1.00,42.21,C
